In [1]:
# SIMULATIONS NOTEBOOK

# This notebook runs a simulation for the Solar Dynamo model with the sABC algorithm.
# The results of the simulation, as well as the parameters used for the simulated data and 
# the simulation itself are all stored in files in the directory Simulations/Simulations i.
# The reason behind this is to have an easier access to the results of already run simulations,
# without running them again.

# NB: In this notebook, there's no function to visualize the results: the visualization is all
# contained in the "visualization.ipynb" notebook.

# RULES:

# There are two ways to use this notebook:
# 1) change all the parameters and then run all -> correct way
# 2) change things randomly and not in order and then run -> wrong way
# Please be careful, some functions change the directory in which everything is being saved; the
# order of the calling of the functions is organized to start from a generic dir, create the dir
# Simulations/Simulation i, go to that directory and then, when everythingis finished, go back to
# the initial dir!!! If you don't do that, it will stay in the subdir and at the next run it will
# create a subdir of a subdir -> if you need to stop midway through because u forgot something, 
# remember to come back to the initial directory (and eliminate the directory that has not correct
# files inside).

# GG EZ - kallo27

In [2]:
# NEEDED PACKAGES -> no visualization!!

using SpecialFunctions
using Distributions
using SimulatedAnnealingABC
using ThreadPinning

In [3]:
# MODEL AND MANAGING FUNCTIONS FILES

include("./model_functions.jl")
include("./managing_functions.jl")

using .model_functions
using .managing_functions

In [4]:
# THREADS SETTINGS AND INFO

ThreadPinning.pinthreads(:cores)
ThreadPinning.threadinfo()


System: 8 cores (no SMT), 8 sockets, 1 NUMA domains

| 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 

# = Julia thread, | = Socket seperator

Julia threads: 8
├ Occupied CPU-threads: 8
└ Mapping (Thread => CPUID): 1 => 0, 2 => 1, 3 => 2, 4 => 3, 5 => 4, ...


In [5]:
# DIRECTORY MANAGING

# Current directory
initial_directory = pwd()

# New directory
create_directory("Simulation")

# NB: After "create_directory", we move to the new directory.
# DON'T RUN THIS AGAIN, wait for the simulation to finish!!!! If you made errors,
# eliminate the Simulations/Simulation i directory and then rerun everything

Directory created at: /home/ubuntu/LCP_B/Project/Simulations/Simulation 16


In [6]:
# SIMULATED DATA PARAMETERS MANAGING

# Parameters for the simulated data
N = 6.2
T = 3.1
tau = 3.5
sigma = 0.04
B_max = 6.0

par = [N, T, tau, sigma, B_max]

# Time
Tsim = [971, 1899]
dt = 0.1

# Writing on file "parameters.csv" of the values set in this cell.
write_params_sim(par, Tsim, dt)

"parameters.csv"

In [7]:
# DATA SIMULATION

# Data simulation with the chosen parameter
sol = bfield(par, Tsim, dt)

# Important solution data that are necessary afterwards
t = sol.t
u = sol[1, :]
du = sol[2, :]

# Creation of the summary statistics from the simulated data
indices = 1:5:120
sim_ss = reduced_fourier_spectrum(u.^2, indices)

# Writing of the solution of the SDDE on file "simulated_sol.csv"
save_solution(sol)

Solution saved to file: /home/ubuntu/LCP_B/Project/Simulations/Simulation 16/simulated_sol.csv


In [8]:
# SIMULATION PARAMETERS MANAGING

# Parameters that can be tuned for new simulations
prior = product_distribution(Uniform(1, 15), Uniform(0.1, 15.0), Uniform(0.1, 6.0), Uniform(0.01, 0.3), Uniform(1, 15))
n_particles = 1000
n_simulation = 50000
v = 1.0
type = 1

# Writing on file "sabc_params.csv" of the values set in this cell.
save_sabc_params(prior, n_particles, n_simulation, v, type, indices)

Parameters saved to: /home/ubuntu/LCP_B/Project/Simulations/Simulation 16/sabc_params.csv


In [9]:
# SIMULATION

# Actual usage of the sABC algorithm
result = sabc(f_dist, prior;
              n_particles = n_particles, 
              n_simulation = n_simulation,
              v = v,
              type = type,
              indices = indices,
              fourier_data = sim_ss, 
              Tsim = Tsim,
              dt = dt)

# Display of the summary of the results
display(result)

# Saving the results to the files: "eps_hist.csv", "u_hist.csv", "rho_hist.csv", "pop_hist.csv".
save_result(result)

┌ Info: Preparing to run SABC algorithm: 'single-epsilon'
└ @ SimulatedAnnealingABC /home/ubuntu/.julia/packages/SimulatedAnnealingABC/e8QsC/src/SimulatedAnnealingABC.jl:188
┌ Info: Using threads: 8 
└ @ SimulatedAnnealingABC /home/ubuntu/.julia/packages/SimulatedAnnealingABC/e8QsC/src/SimulatedAnnealingABC.jl:199
┌ Info: Set BLAS threads = 1 
└ @ SimulatedAnnealingABC /home/ubuntu/.julia/packages/SimulatedAnnealingABC/e8QsC/src/SimulatedAnnealingABC.jl:202
┌ Info: Set 'pinthreads(:cores)' for optimal multi-threading performance
└ @ SimulatedAnnealingABC /home/ubuntu/.julia/packages/SimulatedAnnealingABC/e8QsC/src/SimulatedAnnealingABC.jl:204
┌ Info: Initializing population...
└ @ SimulatedAnnealingABC /home/ubuntu/.julia/packages/SimulatedAnnealingABC/e8QsC/src/SimulatedAnnealingABC.jl:223
┌ Info: Initial resampling (δ = 0.1) - ESS = 996.6799495718246 
└ @ SimulatedAnnealingABC /home/ubuntu/.julia/packages/SimulatedAnnealingABC/e8QsC/src/SimulatedAnnealingABC.jl:277
┌ Info: Population

Approximate posterior sample with 1000 particles:
  - simulations used: 50000
  - average transformed distance: 0.07566
  - ϵ: [0.02882]
  - population resampling: 3
  - acceptance rate: 0.1366
The sample can be accessed with the field `population`.
The history of ϵ can be accessed with the field `state.ϵ_history`.
 -------------------------------------- 


eps_hist.csv data saved to: /home/ubuntu/LCP_B/Project/Simulations/Simulation 16/eps_hist.csv
u_hist.csv data saved to: /home/ubuntu/LCP_B/Project/Simulations/Simulation 16/u_hist.csv
rho_hist.csv data saved to: /home/ubuntu/LCP_B/Project/Simulations/Simulation 16/rho_hist.csv
Posterior parameters saved to: /home/ubuntu/LCP_B/Project/Simulations/Simulation 16/pop.csv
Rho values saved to: /home/ubuntu/LCP_B/Project/Simulations/Simulation 16/rho.csv


In [10]:
# DIRECTORY MANAGING

# We go back to the initial directory
cd(initial_directory)
pwd()

"/home/ubuntu/LCP_B/Project"